<br>

# Introdução

In [ ]:
import os
import math
import time
import shutil
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

In [ ]:
import sys
sys.path.append(os.path.expanduser('~/Documents/Sourcecode'))
from my_vault.credentials_infoaguas import credential

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

<br>

# Functions

<br>

## Cria Driver

In [ ]:
def create_driver(download_dir):
    # Create directory
    [os.makedirs(i, exist_ok=True) for i in [download_dir]]
    
    # Driver Firefox com Profile
    profile = webdriver.FirefoxProfile()
    profile.set_preference('intl.accept_languages', 'pt-BR, pt')
    profile.set_preference('browser.download.folderList', 2)
    profile.set_preference('browser.download.dir', download_dir)
    profile.set_preference('browser.download.manager.showWhenStarting', 'false')
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/octet-stream;application/vnd.ms-excel;text/html')
    profile.update_preferences()

    # Driver Firefox com Options
    options = Options()
    options.headless = False

    # Driver
    return webdriver.Firefox(firefox_profile=profile, options=options)

<br>

## Go Login

In [ ]:
def go_login(payload):
    try:
        url = 'https://sistemainfoaguas.cetesb.sp.gov.br/'
        driver.get(url)
        time.sleep(1)

        # Login
        username = driver.find_element_by_id('Email')
        username.send_keys(payload['username'])
        time.sleep(1)

        password = driver.find_element_by_id('Senha')
        password.send_keys(payload['password'])
        time.sleep(1)

        driver.find_element_by_xpath("//button[@onclick=\"entrarClick();\"]").click()
        time.sleep(1)

    except:
        print('Talvez já estivesse logado!?')

<br>

## Go Relatório Superficial

In [ ]:
def go_rel_superficiais():
    # Relatório de Águas Superficiais
    url = 'https://sistemainfoaguas.cetesb.sp.gov.br/AguasSuperficiais/RelatorioQualidadeAguasSuperficiais'
    driver.get(url)
    driver.find_element_by_xpath("//button[@type=\"submit\"]").click()

<br>

## Get Table

Monta Tabela com Todos os Pontos

In [ ]:
def get_df():
    # Número de Páginas
    driver.find_element_by_xpath("//*[@name='gridConsulta_length']//option[text()='100']").click()
    info = driver.find_element_by_xpath("//*[@id='gridConsulta_info']").text
    n_pages = math.ceil(int(info.split(' ')[-2])/100)

    dfs = []
    for i in range(0, n_pages):
        soup = BeautifulSoup(driver.page_source)
        soup_table = soup.find('table')
        df = pd.read_html(str(soup_table))
        df = df[0]
        dfs.append(df)
        driver.find_element_by_xpath("//*[@id='gridConsulta_next']").click()
        time.sleep(1)

    # Concatena
    df = pd.concat(dfs)
    df['Data Início'] = pd.to_datetime(df['Data Início'], format='%d/%m/%Y', errors='coerce')
    df['Data Fim'] = pd.to_datetime(df['Data Fim'], format='%d/%m/%Y', errors='coerce')
    df.drop(['Unnamed: 6'], axis=1, errors='ignore', inplace=True)
    return df

<br>

## Set Ponto

Função que pesquisa um ponto

In [ ]:
def set_ponto(ponto):
    # Clica em Consulta
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@aria-controls='gridConsulta']")))
    time.sleep(1)
    
    # Procura Ponto
    search = driver.find_element_by_xpath("//*[@aria-controls='gridConsulta']")
    search.clear()
    search.send_keys(ponto)
    time.sleep(1)
    
    # Pesquisa
    driver.find_element_by_xpath("//*[@id='CodigoPonto']").click()
    time.sleep(1)

<br>

## Set Dates

In [ ]:
def set_date(tipo_data, data):
    #tipo_data = 'DataInicial'
    #tipo_data = 'DataFinal'
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@class='btn btn-primary']")))

    meses = {
        1: 'Jan',
        2: 'Fev',
        3: 'Mar',
        4: 'Abr',
        5: 'Mai',
        6: 'Jun',
        7: 'Jul',
        8: 'Ago',
        9: 'Set',
        10: 'Out',
        11: 'Nov',
        12: 'Dez',    
    }

    driver.find_element_by_xpath("//*[@id='" + tipo_data + "']").click()
    driver.find_element_by_xpath(
        "//*[@class='datepicker']//*[@class='datepicker-days']//*[@class='picker-switch']").click()

    # Get Year
    year = driver.find_element_by_xpath(
        "//*[@class='datepicker']//*[@class='datepicker-months']//*[@class='picker-switch']")
    year_n = int(year.text)
    while year.text != str(data.year):        
        prev_year = driver.find_element_by_xpath(
            "//*[@class='datepicker']//*[@class='datepicker-months']//*[@class='prev']").click()
        year = driver.find_element_by_xpath(
            "//*[@class='datepicker']//*[@class='datepicker-months']//*[contains(@class, 'picker-switch')]")
        year_n = int(year.text)

    # Get Month
    months = driver.find_elements_by_xpath(
        "//*[@class='datepicker']//*[@class='datepicker-months']//*[contains(@class, 'month')]")
    for month in months:
        if month.text == meses[data.month]:
            month_n = int(list(meses.keys())[list(meses.values()).index(month.text)])
            month.click()
            break

    # Get Days
    days = driver.find_elements_by_xpath(
        "//*[@class='datepicker-days']//*[contains(@class, 'day')]")
    for day in days:
        if day.text == str(data.day):
            day_n = int(day.text)
            day.click()
            break
    
    # Message
    if date(year_n, month_n, day_n) == data:
        print('> {} {} definida'.format(
            tipo_data.replace('Data', 'Data '),
            data.strftime('%d.%m.%Y')))
    else:
        print('> Deu ruim!')
    
    time.sleep(1)
    return

In [ ]:
def set_both_dates(startdate, enddate):
    set_date('DataInicial', startdate)
    set_date('DataFinal', enddate)

<br>

## Set Download

In [ ]:
def set_download_file(directory, ponto, startdate, enddate):    
    # Define o novo nome
    filename_new = '{} {} {}.xlsx'.format(
        ponto,
        startdate.strftime('%Y-%m-%d'),
        enddate.strftime('%Y-%m-%d'))

    # Clica para Gerar Planilha
    time.sleep(1)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@class='btn btn-primary']"))).click()
    time.sleep(3)
    
    # Procura mensagem
    try:
        msg = driver.find_element_by_xpath("//*[@class='field-validation-error alert alert-danger']").text
    except:
        msg = ''
    
    if msg == 'Não há informações para o ponto e período selecionados':
        # Clica em Voltar
        driver.find_element_by_xpath("//*[@class='btn btn-default']").click()
        print('>> {}: {} - {}'.format(
            msg,
            startdate.strftime('%d.%m.%Y'),
            enddate.strftime('%d.%m.%Y')))
        
    # Faz o download e renomeia
    elif msg == '':
        # Artificio para ver se já voltou para a opção da consulta
        WebDriverWait(driver, 30).until(EC.presence_of_element_located(
            (By.XPATH, "//*[@aria-controls='gridConsulta']")))
        
        # Se tem o arquivo
        teste = os.path.isfile(os.path.join(directory, 'RelatorioQualidadeAguasSuperficiais.xlsx'))
        while teste:
            shutil.move(
                max([os.path.join(directory, file) for file in os.listdir(directory)], key=os.path.getctime),
                os.path.join(directory, filename_new))
            time.sleep(1)
            teste = os.path.isfile(os.path.join(directory, 'RelatorioQualidadeAguasSuperficiais.xlsx'))

<br>

## Loop Stations

In [ ]:
def loop_stations(df, download_dir):
    # Set Date Min
    date_min = min(df['Data Início'])

    # Loop
    for index, row in df.iterrows():
        # Define: Ponto
        ponto_mon = row['Cód.Ponto']

        # Define: Data Inicial
        if pd.isnull(row['Data Início']):
            startdate = date(date_min.year, 1, 1)

        elif not(pd.isnull(row['Data Início'])):
            startdate = date(row['Data Início'].year, 1, 1)

        # Define: Data Final
        if pd.isnull(row['Data Fim']):
            enddate = date.today()

        elif not(pd.isnull(row['Data Fim'])):            
            enddate = date(row['Data Fim'].year, 12, 31)

        # Message
        print('{}: {} - {}'.format(ponto_mon, startdate.year, enddate.year))            

        # Número de Anos
        n_years = relativedelta(enddate, startdate).years

        # New
        limit_years = 4
        for passo in range(0, math.ceil(n_years/5)):
            # Define uma Data Final com avanço de X anos
            enddate_loop = date((startdate + relativedelta(years=limit_years)).year, 12, 31)
            if enddate_loop > enddate:
                enddate_loop = enddate

            # Seleciona o Ponto
            set_ponto(ponto_mon)

            # Definir Datas
            set_both_dates(startdate, enddate_loop)

            # Gerar Planilha
            set_download_file(download_dir, ponto_mon, startdate, enddate_loop)

            # Define uma Data Inicial com avanço de X anos
            startdate = startdate + relativedelta(years=limit_years + 1)
            time.sleep(3)

<br>

# Run

<br>

## Monta Tabela com Lista de Pontos de Monitoramento

In [ ]:
driver = create_driver(os.getcwd())

In [ ]:
go_login(credential)
go_rel_superficiais()

df = get_df()
driver.quit()

df

<br>

## Salva Tabela

In [ ]:
df.to_csv(
    os.path.join('data', 'tab_pontos_monitoramento.csv'),
    index=False
)

<br>

## Individual

In [ ]:
# Cria driver com 'download path' definido
download_dir = os.path.join(os.getcwd(), 'data', 'A')
driver = create_driver(download_dir)

# Faz o login e vai para Relatório de Águas Superficiais
go_login(credential)
go_rel_superficiais()

# Escolhe um Ponto de Monitoramento
cod_ponto = 'DADO02600' # Deu erro em ABCA02300
set_ponto(cod_ponto)

# Escolhe uma data de Início e Fim
startdate = date(2007, 1, 1)
enddate = date(2011, 12, 31)
set_both_dates(startdate, enddate)

# Faz Download
set_download_file(download_dir, cod_ponto, startdate, enddate)

In [ ]:
driver.quit()

<br>

## Vários, por letra

In [ ]:
df = pd.read_csv(
    os.path.join('data', 'tab_pontos_monitoramento.csv'),
)

df

In [ ]:
# Exclui um determinado ponto da lista
#df = df[df['Cód.Ponto']!='SDOM04600']

# Seleciona os pontos que começam com determinado conjunto de letras
df = df[df['Cód.Ponto'].str.startswith('TURV')]
df

In [ ]:
list_letter = list(set(df['Cód.Ponto'].astype(str).str[0]))
list_letter.sort()
list_letter = list_letter[4:]
list_letter

In [ ]:
for letter in list_letter:
    # Cria Driver
    download_dir = os.path.join(os.getcwd(), 'data', letter)
    driver = create_driver(download_dir)
    
    # Seleciona tabela
    df_letter = df[df['Cód.Ponto'].str.startswith(letter)]
    print(df_letter.head(2))
    
    # Para cada letra, faz login!
    go_login(credential)
    go_rel_superficiais()
    
    # Loop
    loop_stations(df_letter, download_dir)
    
    # Sair do Driver
    driver.quit()